In [8]:
# 05_cleaning_artworks.ipynb

import sys
import os
import pandas as pd
import numpy as np
import re

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.data_loader as dl

# --- Load data ---
df = dl.load_artworks()

# 1. Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# 2. Remove duplicates
df.drop_duplicates(inplace=True)

# 3. Drop completely empty rows and columns
df.dropna(how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

# 4. Fill key missing values
df['artist'].fillna('Unknown', inplace=True)
df['nationality'].fillna('Unknown', inplace=True)
df['gender'].fillna('Unknown', inplace=True)
df['medium'].fillna('Unknown', inplace=True)
df['date'].fillna('Unknown', inplace=True)
df['dateacquired'].fillna('Unknown', inplace=True)

# 5. Convert date-related columns to numeric or datetime
df['begindate'] = pd.to_numeric(df['begindate'], errors='coerce')
df['enddate'] = pd.to_numeric(df['enddate'], errors='coerce')
df['dateacquired'] = pd.to_datetime(df['dateacquired'], errors='coerce')

# 5.1 Extract year from 'date' field
def extract_year(text):
    if pd.isna(text) or text == 'Unknown':
        return np.nan
    match = re.search(r"\b(\d{4})\b", str(text))
    return int(match.group(1)) if match else np.nan

df['year'] = df['date'].apply(extract_year)

# 6. Fix 'gender' values
df['gender'] = df['gender'].str.strip().str.capitalize()
df['gender'] = df['gender'].replace({'': 'Unknown', 'Nan': 'Unknown'})

# 7. Normalize 'nationality'
df['nationality'] = df['nationality'].str.strip().str.title()
df['nationality'].replace('', 'Unknown', inplace=True)

# 8. Ensure 'objectid' is unique
df.drop_duplicates(subset='objectid', inplace=True)

# 9. Reset index
df.reset_index(drop=True, inplace=True)

# 10. Summary
print("Cleaned dataset shape:", df.shape)
print("Missing values per column:\n", df.isnull().sum())

# Optional: Save cleaned dataset
df.to_csv("../outputs/artworks_cleaned.csv", index=False)


Cleaned dataset shape: (158909, 30)
Missing values per column:
 title                     39
artist                     0
constituentid           1265
artistbio               6737
nationality                0
begindate             158909
enddate               158909
gender                     0
date                       0
medium                     0
dimensions              8634
creditline              1529
accessionnumber            0
classification             1
department                 0
dateacquired            5619
cataloged                  0
objectid                   0
url                    58280
imageurl               67292
onview                157621
circumference_(cm)    158899
depth_(cm)            140598
diameter_(cm)         157510
height_(cm)            30205
length_(cm)           158185
weight_(kg)           158612
width_(cm)             31063
duration_(sec.)       156953
year                    5500
dtype: int64
